In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
import pickle

%reload_ext autoreload
%autoreload 2
from config import keys
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns


# University of Copenhagen Dataset

In [3]:
df_1 = pd.read_csv('data/Original/NAACL_SRW_2016.csv')
df_1.head()

,572342978255048705,racism
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [4]:
df_1.columns = ['id', 'label']
df_1.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [5]:
df_racsex = df_1[(df_1['label']=='racism') | (df_1['label']=='sexism')]

In [6]:
df_normal = df_1[(df_1['label']=='none')]

In [7]:
def group_list(lst, size=100):
    new_list =[]
    idx = 0
    while idx < len(lst):        
        new_list.append(','.join([str(item) for item in lst[idx:idx+size]]))
        idx += size
    return new_list

def tweets_request(tweets_ids):
    lst = []
    for batch in tqdm(tweets_ids):
        url = "https://api.twitter.com/2/tweets?ids={}&&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username".format(batch)
        payload={}
        headers = {'Authorization': 'Bearer ' + keys['bearer_token'],
        'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}
        r = requests.request("GET", url, headers=headers, data=payload)
        data = r.json()
        if 'data' in data.keys():
            lst.append(pd.DataFrame(data['data']))
    return pd.concat(lst)


In [8]:
racsex_id = group_list(list(df_racsex.id))

In [9]:
normal_id = group_list(list(df_normal.id))

In [10]:
df_rac_sex = tweets_request(racsex_id)

100%|██████████| 54/54 [00:38<00:00,  1.41it/s]


In [11]:
df_rac_sex['label'] = 1
df_1_hate= df_rac_sex[['text', 'label']]

In [12]:
df_normal = tweets_request(normal_id)

100%|██████████| 116/116 [01:29<00:00,  1.30it/s]


In [13]:
df_1_hate.head()

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
0,RT @YesYoureRacist: At least you're only a tin...,1


In [15]:
df_normal['label'] = 0
df_1_normal = df_normal[['text', 'label']]

In [16]:
df_1_normal.head()

,text,label
0,Someone is going home #mkr ...that obviously c...,0
1,They didn't even wash the chicken 😩 #MKR,0
2,#MKR Is honestly so fucking staged. The most o...,0
3,So bummed the dull chicks on #MKR are using su...,0
4,Can someone smash that bottle of Rose &amp; Li...,0


In [17]:
frames = [df_1_hate, df_1_normal]
  
result = pd.concat(frames)
display(result)

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
0,RT @YesYoureRacist: At least you're only a tin...,1
...,...,...
66,@ProoN ❤️,0
67,RT @Transition: @freebsdgirl just so I’m clear...,0
68,RT @Leonard_Delaney: @freebsdgirl I know! Holy...,0
69,"FLOSS Weekly, open source projects, and paying...",0


In [18]:
result.to_csv("data/Original/Copenhagen_Dataset.csv")

# Davidson Dataset

In [19]:
df_2 = pd.read_csv("data/Original/labeled_data.csv", index_col=0)
df_2.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [20]:
df_2 = df_2.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'], axis=1)
df_2.columns = ['label', 'text']
df_2.label.value_counts()

1    19190
2     4163
0     1430
Name: label, dtype: int64

In [21]:
df_2 = df_2.replace({'label': {1: 0, 2: 0, 0: 1}})
df_2.label.value_counts()

0    23353
1     1430
Name: label, dtype: int64

In [22]:
df_2.head()

,label,text
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [23]:
df_2.count()

label    24783
text     24783
dtype: int64

In [24]:
df_2.to_csv("data/Original/Davidson_Dataset.csv")

# Aristotle Dataset

In [37]:
df_3 = pd.read_csv("data/Original/hatespeechtwitter.csv")
df_3.head()

,tweet_id,maj_label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal


In [38]:
df_3.columns = ['id', 'label']
df_3.label.value_counts()

normal     52835
spam       13404
abusive    10122
hateful     3635
Name: label, dtype: int64

In [39]:
# setup url and headers for calling Twitter API as per documentation
url = "https://api.twitter.com/2/tweets?ids=847661947159891972,847799130277675008,848933211375779840&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username"
payload={}
headers = {'Authorization': 'Bearer ' + keys['bearer_token'], 'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}

# make GET request to Twitter API to get response object 
r = requests.request("GET", url, headers=headers, data=payload)

# convert payload to dictionary
data = r.json()

In [40]:
df_hateful = df_3[df_3['label']=='hateful']
hate_ids = group_list(list(df_hateful.id))

In [41]:
df_normal = df_3[df_3['label']=='normal']
normal_ids = group_list(list(df_normal.id))

In [42]:
df_hate = tweets_request(hate_ids)

100%|██████████| 37/37 [00:15<00:00,  2.33it/s]


In [43]:
df_normal = tweets_request(normal_ids)

100%|██████████| 529/529 [02:53<00:00,  3.05it/s]


In [44]:
df_hate['label'] = 1
df_3_hate = df_hate[['text', 'label']]
df_3_hate.count()

text     1607
label    1607
dtype: int64

In [45]:
df_normal['label'] = 0
df_3_normal = df_normal[['text', 'label']]
df_3_normal.count()

text     14298
label    14298
dtype: int64

In [46]:
frames = [df_3_hate, df_3_normal]
  
result = pd.concat(frames)
display(result)

,text,label
0,I hate er chase because if the Bitch that work...,1
1,RT @nyctophil3: Pineapples do not belong on pi...,1
2,Niggas keep talking about women wearing weave ...,1
3,@vappywave idiot that's not gonna work. you go...,1
4,RT @ayevonnn: bruh i fucking hate people like ...,1
...,...,...
54,Industry – Government Collaboration Will Make ...,0
55,@RAIL Indeed that huge plume of smoke across a...,0
56,"@JaredStillman 2015 yes, we lost 1st round to ...",0
57,@JoeShurmaitis I didn't know #carrier #ford# ...,0


In [48]:
result.to_csv("data/Original/Aristotle_Dataset.csv")

# HASOC Dataset

In [50]:
df_4 = pd.read_csv("data/Original/english_dataset.tsv", delimiter = "\t")
df_4.head(20)

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE
6,hasoc_en_7,@ICC Latest design of #WC2019 trophy. #CWC2019...,NOT,NONE,NONE
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN
8,hasoc_en_9,Thanks for your support! Wow 600k. Graffiti ha...,NOT,NONE,NONE
9,hasoc_en_10,By wearing the #BalidaanBadge over his gloves ...,NOT,NONE,NONE


In [52]:
df_4_hate = df_4[df_4.task_2 == 'HATE']
df_4_hate['label'] = 1
df_4_hate= df_4_hate[['text', 'label']]
df_4_hate.head()

,text,label
1,@politico No. We should remember very clearly ...,1
11,I don’t know how much more I can take! 45 is a...,1
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1
27,Disgusting! And they don’t change their cultur...,1
34,"@TheRealOJ32 Of all the retired NFL players, w...",1


In [57]:
df_4_normal = df_4.loc[(df_4['task_1']=="NOT") & (df_4['task_2']=="NONE") & (df_4['task_1']=="NOT")]
df_4_normal.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE


In [58]:
df_4_normal['label'] = 0
df_4_normal= df_4_normal[['text', 'label']]
df_4_normal.head()

,text,label
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
2,@cricketworldcup Guess who would be the winner...,0
3,Corbyn is too politically intellectual for #Bo...,0
4,All the best to #TeamIndia for another swimmin...,0
5,@kellymiller513 @TheRealOJ32 I hope you rememb...,0


In [59]:
frames = [df_4_hate, df_4_normal]
  
result = pd.concat(frames)
display(result)

,text,label
1,@politico No. We should remember very clearly ...,1
11,I don’t know how much more I can take! 45 is a...,1
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1
27,Disgusting! And they don’t change their cultur...,1
34,"@TheRealOJ32 Of all the retired NFL players, w...",1
...,...,...
5824,@DOlusegun @elrufai Hope they don't becomes #D...,0
5825,@gtconway3d @KellyannePolls @realDonaldTrump H...,0
5827,@washingtonpost This is something everyone sho...,0
5828,Sorry about that. Has anybody got a #dickhead ...,0


In [60]:
result.to_csv("data/Original/HASOC_Dataset.csv")